In [4]:
#%pip install pandas scikit-learn sqlite3

import pandas as pd
import numpy as np
import os
import sqlite3
from sklearn.preprocessing import StandardScaler
from tabulate import tabulate

def transform_and_store_data(input_filename="../5.DataPreparation/cleaned_data.csv", db_filename="bank_churn.db"):
    """
    Performs data transformation and stores the final dataset in a SQLite database.

    Args:
        input_filename (str): The path to the cleaned data CSV file.
        db_filename (str): The name of the SQLite database file.
    """

    # Check if the input file exists
    if not os.path.exists(input_filename):
        print(f"Error: The file '{input_filename}' was not found. Please ensure the cleaned data is generated.")
        return

    print("--- Starting Data Transformation and Storage ---")

    # Load the cleaned data
    # Assuming the cleaned data is available in the environment as a DataFrame
    # If you saved it to a CSV, load it from there:
    #df = pd.read_csv(input_filename)
    # For this example, we'll use the clean_dataset DataFrame directly
    try:
        #df = clean_dataset.copy() # Use the DataFrame from the previous step
        df = pd.read_csv(input_filename)
    except NameError:
        print("Error: The 'clean_dataset' DataFrame was not found. Please run the previous cell first.")
        return


    # Create a table for process logging
    process_log = []

    def add_log(step, action, result):
        process_log.append([step, action, result])

    add_log("Step 1", "Loading Data", f"Successfully loaded {len(df)} records from the environment.")

    # Perform transformations for feature engineering
    print("\nStep 2: Performing Transformations for Feature Engineering")
    add_log("Step 2", "Feature Engineering", "In progress...")

    # Create new features using the new column names
    df['TenureToAgeRatio'] = df['num__Tenure'] / df['num__Age']
    df['IsEngaged'] = df.apply(
        lambda row: 1 if row['num__NumOfProducts'] > 1 and row['num__IsActiveMember'] == 1 else 0,
        axis=1
    )

    # Scale and normalize numerical features using the new column names
    numeric_features = ['num__CreditScore', 'num__Age', 'num__Balance', 'num__EstimatedSalary', 'TenureToAgeRatio']
    scaler = StandardScaler()
    df[numeric_features] = scaler.fit_transform(df[numeric_features])

    add_log("Step 2", "Feature Engineering", "Completed. New features created and numerical data scaled.")

    # Store the transformed data in a relational database
    print("\nStep 3: Storing Data in a Relational Database (SQLite)")
    add_log("Step 3", "Data Storage", "In progress...")

    conn = sqlite3.connect(db_filename)
    df.to_sql('transformed_data', conn, if_exists='replace', index=False)
    conn.close()

    add_log("Step 3", "Data Storage", f"Completed. Data stored in '{db_filename}' in table 'transformed_data'.")

    print("\n--- Summary of Transformation and Storage Steps ---")
    print(tabulate(process_log, headers=["Step", "Action", "Result"], tablefmt="pipe"))

    # Deliverables:
    print("\n--- Deliverables ---")

    # SQL schema design
    print("\n**1. SQL Schema Design**")
    print(f"""
-- SQL Schema for the 'transformed_data' table in '{db_filename}'
CREATE TABLE transformed_data (
    "num__CreditScore" REAL,
    "num__Age" REAL,
    "num__Tenure" REAL,
    "num__Balance" REAL,
    "num__NumOfProducts" REAL,
    "num__HasCrCard" INTEGER,
    "num__IsActiveMember" INTEGER,
    "num__Exited" INTEGER,
    "cat__Geography_France" INTEGER,
    "cat__Geography_Germany" INTEGER,
    "cat__Geography_Spain" INTEGER,
    "cat__Gender_Female" INTEGER,
    "cat__Gender_Male" INTEGER,
    "TenureToAgeRatio" REAL,
    "IsEngaged" INTEGER
);
""")

    # Sample queries to retrieve transformed data
    print("**2. Sample Queries**")
    print("""
-- Query 1: Retrieve the top 5 customers with the highest scaled CreditScore
SELECT
    "num__CreditScore",
    "num__Age",
    "num__Exited"
FROM
    transformed_data
ORDER BY
    "num__CreditScore" DESC
LIMIT 5;

-- Query 2: Calculate the average scaled balance for customers who exited vs. stayed
SELECT
    "num__Exited",
    AVG("num__Balance") as avg_scaled_balance
FROM
    transformed_data
GROUP BY
    "num__Exited";
""")

    # Summary of transformation logic applied
    print("**3. Summary of Transformation Logic**")
    print("""
- **Feature Engineering:**
    - `TenureToAgeRatio`: A new numerical feature created by dividing `num__Tenure` by `num__Age`.
    - `IsEngaged`: A binary feature derived from `num__NumOfProducts` and `num__IsActiveMember` to indicate high customer engagement.
- **Feature Scaling:**
    - `StandardScaler` was applied to all numerical features (`num__CreditScore`, `num__Age`, `num__Balance`, `num__EstimatedSalary`, `TenureToAgeRatio`). This standardizes the data by removing the mean and scaling to unit variance. This is essential for algorithms sensitive to feature magnitudes.
- **Data Storage:**
    - The final DataFrame was stored in a local SQLite database named 'bank_churn.db' in a table called 'transformed_data'.
""")

if __name__ == "__main__":
    transform_and_store_data()


--- Starting Data Transformation and Storage ---

Step 2: Performing Transformations for Feature Engineering

Step 3: Storing Data in a Relational Database (SQLite)

--- Summary of Transformation and Storage Steps ---
| Step   | Action              | Result                                                                 |
|:-------|:--------------------|:-----------------------------------------------------------------------|
| Step 1 | Loading Data        | Successfully loaded 14950 records from the environment.                |
| Step 2 | Feature Engineering | In progress...                                                         |
| Step 2 | Feature Engineering | Completed. New features created and numerical data scaled.             |
| Step 3 | Data Storage        | In progress...                                                         |
| Step 3 | Data Storage        | Completed. Data stored in 'bank_churn.db' in table 'transformed_data'. |

--- Deliverables ---

**1. SQL Schema D